In [122]:
import pandas as pd
import os
os.listdir('data')

['.ipynb_checkpoints',
 '2024_District_Details.xlsx',
 '2024_DISTRICT_GRAD_RATE.xlsx',
 '22_TAX_INCOME.xlsx',
 '23-24_Achievement_District.xlsx',
 '2324_DISTRICT_SPEND_PER_PUPIL.xlsx',
 '2324_DIST_RACE_DIS.xlsx',
 '2324_VA_DIST_DETAILS.xlsx',
 'DIST_PROPERTY_TAX.xlsx']

In [123]:
DIST_VA = pd.read_excel('data/2324_VA_DIST_DETAILS.xlsx', sheet_name=1,index_col=0)
DIST_GRAD_RATE = pd.read_excel('data/2024_DISTRICT_GRAD_RATE.xlsx', sheet_name=1,index_col=0)
DIST_RACE_DIS = pd.read_excel('data/2324_DIST_RACE_DIS.xlsx', sheet_name=1,index_col=0)
DIST_DETAIL = pd.read_excel('data/2024_District_Details.xlsx', sheet_name=1,index_col=0)
DIST_ACHEIVE = pd.read_excel('data/23-24_Achievement_District.xlsx', sheet_name=1,index_col=0)
DIST_SPEND = pd.read_excel('data/2324_DISTRICT_SPEND_PER_PUPIL.xlsx', sheet_name=1,index_col=0)
DIST_INCOME_TAX = pd.read_excel('data/22_TAX_INCOME.xlsx', sheet_name=1,index_col=3)
DIST_PROPERTY_TAX = pd.read_excel('data/DIST_PROPERTY_TAX.xlsx', sheet_name=1,index_col=2)

C:\Users\kdelv\miniconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
C:\Users\kdelv\miniconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [124]:
# Clean Distric Value Added DF
DIST_VA = DIST_VA.drop(columns = 'Watermark')

In [125]:
# Clean Distric Graduation Rate DF
DIST_GRAD_RATE = DIST_GRAD_RATE.drop(columns=['District Name','County','Region','Watermark'])

In [126]:
# Clean Distric Detail DF
import pandas as pd
import numpy as np

def transform_district_data(df: pd.DataFrame) -> pd.DataFrame:
    data = df.copy()

    if data.index.name:
        district_id_col = data.index.name
    else:
        # Fallback if the index is not named
        district_id_col = 'District ID'
        data.index.name = district_id_col

    data.reset_index(inplace=True)

    data['Enrollment'] = pd.to_numeric(data['Enrollment'], errors='coerce')
    data['Attendance Rate'] = pd.to_numeric(data['Attendance Rate'], errors='coerce')

    pivoted_data = data.pivot_table(
        index=district_id_col,
        columns='Student Group',
        values='Enrollment'
    ).fillna(0).astype(int)

    all_students_df = data[data['Student Group'] == 'All Students'].set_index(district_id_col)
    total_attendance_map = all_students_df['Attendance Rate']

    if 'All Students' in pivoted_data.columns:
        pivoted_data.rename(columns={'All Students': 'Total Students'}, inplace=True)

    pivoted_data['Total Attendance Percent'] = pivoted_data.index.map(total_attendance_map)

    if 'Total Students' in pivoted_data.columns:
        student_group_cols = [col for col in pivoted_data.columns if col not in ['Total Students', 'Total Attendance Percent']]
        final_order = ['Total Students', 'Total Attendance Percent'] + student_group_cols
        pivoted_data = pivoted_data[final_order]

    return pivoted_data.reset_index()
DIST_DETAIL = transform_district_data(DIST_DETAIL)
DIST_DETAIL.set_index('District IRN', inplace=True)


In [127]:
DIST_ACHEIVE = DIST_ACHEIVE.drop(columns=['District Name','County','Region','Watermark','Maximum District Performance Index Score 2023-2024'])

In [128]:
# Clean District Spend
DIST_SPEND = DIST_SPEND.drop(columns=['District Name','County','Region','Watermark','State-Level Expenditures per Equivalent Pupil','State-Level Expenditures per Equivalent Pupil - Federal Funds','State-Level Expenditures per Equivalent Pupil - State and Local Funds'])

In [129]:
DIST_INCOME_TAX.info()
DIST_INCOME_TAX.drop(columns=['COUNTY','SCHOOL DISTRICT','PUN','SCHOOL DISTRICT NUMBER'],inplace=True)

<class 'pandas.core.frame.DataFrame'>
Index: 609 entries, 61903 to 45625
Data columns (total 20 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   COUNTY                                     609 non-null    object 
 1   SCHOOL DISTRICT                            609 non-null    object 
 2   PUN                                        609 non-null    int64  
 3   SCHOOL DISTRICT NUMBER                     609 non-null    int64  
 4   NUMBER OF RETURNS                          609 non-null    int64  
 5   NUMBER OF PERSONAL EXEMPTIONS              609 non-null    int64  
 6   TOTAL FEDERAL ADJUSTED GROSS INCOME        609 non-null    float64
 7   AVERAGE FEDERAL ADJUSTED GROSS INCOME      609 non-null    float64
 8   ADJUSTED GROSS INCOME RANK                 609 non-null    int64  
 9   MEDIAN FEDERAL ADJUSTED GROSS INCOME       609 non-null    float64
 10  MEDIAN FEDERAL ADJUSTED G

In [130]:
DIST_PROPERTY_TAX.info()
DIST_PROPERTY_TAX.drop(columns=['County','School District Number','PUN','School District Name','Rank'],inplace=True)

<class 'pandas.core.frame.DataFrame'>
Index: 611 entries, 61903 to 45625
Data columns (total 14 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   County                                          611 non-null    object 
 1   School District Number                          611 non-null    int64  
 2   PUN                                             611 non-null    int64  
 3   School District Name                            611 non-null    object 
 4   Real Property Taxable Value (a)                 611 non-null    int64  
 5   Real Property Taxes Charged                     611 non-null    int64  
 6   Public Utility Tangible Property Taxable Value  611 non-null    int64  
 7   Public Utility Tangible Property Taxes Levied   611 non-null    int64  
 8   Total Value                                     611 non-null    int64  
 9   Total Taxes                               

In [131]:
# Join Dataframes
combined_df = pd.merge(DIST_VA,DIST_GRAD_RATE, on='District IRN')
combined_df = pd.merge(combined_df,DIST_DETAIL, on='District IRN')
combined_df = pd.merge(combined_df,DIST_ACHEIVE, on='District IRN')
combined_df = pd.merge(combined_df,DIST_SPEND, on='District IRN')
combined_df = pd.merge(combined_df,DIST_INCOME_TAX, on='District IRN')
combined_df = pd.merge(combined_df,DIST_PROPERTY_TAX, on='District IRN')

In [132]:
#drop invalid columns
combined_df.drop(48975,inplace=True)
combined_df[combined_df['Overall Composite'] == 'NC']

,District Name,County,Region,Progress Component Star Rating,Overall Composite,Overall Effect Size,Graduation Rate Component Rating,Graduation Rate Component Percent (Weighted Graduation Rate),Four Year Graduation Rate - Class of 2023,Four Year Graduation Rate Numerator - Class of 2023,...,NUMBER OF SENIOR CITIZEN CREDITS,Real Property Taxable Value (a),Real Property Taxes Charged,Public Utility Tangible Property Taxable Value,Public Utility Tangible Property Taxes Levied,Total Value,Total Taxes,Real Property,Public Utility Tangible Property,Amount
District IRN,,,,,,,,,,,,,,,,,,,,,


In [133]:
#convert Numbers
combined_df['Overall Composite'] = combined_df['Overall Composite'].astype(float)
combined_df['Overall Effect Size'] = combined_df['Overall Effect Size'].astype(float)
combined_df['Graduation Rate Component Percent (Weighted Graduation Rate)'] = combined_df['Graduation Rate Component Percent (Weighted Graduation Rate)'].astype(float)
combined_df['Four Year Graduation Rate - Class of 2023'] = combined_df['Four Year Graduation Rate - Class of 2023'].astype(float)
combined_df['Four Year Graduation Rate Numerator - Class of 2023'] = combined_df['Four Year Graduation Rate Numerator - Class of 2023'].astype(float)
combined_df['Four Year Graduation Rate Denominator - Class of 2023'] = combined_df['Four Year Graduation Rate Denominator - Class of 2023'].astype(float)
combined_df['Five Year Graduation Rate - Class of 2022'] = combined_df['Five Year Graduation Rate - Class of 2022'].astype(float)
combined_df['Five Year Graduation Rate Numerator - Class of 2022'] = combined_df['Five Year Graduation Rate Numerator - Class of 2022'].astype(float)
combined_df['Five Year Graduation Rate Denominator - Class of 2022'] = combined_df['Five Year Graduation Rate Denominator - Class of 2022'].astype(float)

In [134]:
#convert Money and Stars
combined_df['Expenditures per Equivalent Pupil'] = combined_df['Expenditures per Equivalent Pupil'].str.replace(r'[$,€£]', '', regex=True).str.replace(',', '').astype(float)
combined_df['Expenditures per Equivalent Pupil - Federal Funds'] = combined_df['Expenditures per Equivalent Pupil - Federal Funds'].str.replace(r'[$,€£]', '', regex=True).str.replace(',', '').astype(float)
combined_df['Expenditures per Equivalent Pupil - State and Local Funds'] = combined_df['Expenditures per Equivalent Pupil - State and Local Funds'].str.replace(r'[$,€£]', '', regex=True).str.replace(',', '').astype(float)

#Stars

combined_df['Progress Component Star Rating'] = combined_df['Progress Component Star Rating'].str.replace(' Stars','', regex=True).str.replace(' Star','',regex=True).astype(int)
combined_df['Graduation Rate Component Rating'] = combined_df['Graduation Rate Component Rating'].str.replace(' Stars','', regex=True).str.replace(' Star','',regex=True).astype(int)
combined_df['Achievement Component Star Rating'] = combined_df['Achievement Component Star Rating'].str.replace(' Stars','', regex=True).str.replace(' Star','',regex=True).astype(int)


combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 606 entries, 442 to 139303
Data columns (total 68 columns):
 #   Column                                                         Non-Null Count  Dtype  
---  ------                                                         --------------  -----  
 0   District Name                                                  606 non-null    object 
 1   County                                                         606 non-null    object 
 2   Region                                                         606 non-null    object 
 3   Progress Component Star Rating                                 606 non-null    int64  
 4   Overall Composite                                              606 non-null    float64
 5   Overall Effect Size                                            606 non-null    float64
 6   Graduation Rate Component Rating                               606 non-null    int64  
 7   Graduation Rate Component Percent (Weighted Graduation Rate)  

In [135]:
# Effective Property Tax Rate
combined_df['Effective Property Tax Rate'] = (combined_df['Real Property Taxes Charged'] / combined_df['Real Property Taxable Value (a)']) * 100

# Effective Income Tax Rate
combined_df['Effective Income Tax Rate'] = (combined_df['TOTAL OHIO INCOME TAX LIABILITY'] / combined_df['TOTAL OHIO INCOME TAX BASE']) * 100

# Total Local Tax Burden (Overall Local Effective Rate)
combined_df['Total Local Tax Burden'] = (combined_df['Total Taxes'] / combined_df['Total Value']) * 100

# Per Capita Income Tax Liability
combined_df['Per Capita Income Tax Liability'] = combined_df['TOTAL OHIO INCOME TAX LIABILITY'] / combined_df['NUMBER OF RETURNS']

# Student to Taxpayer Ratio
combined_df['Student to Taxpayer Ratio'] = combined_df['Total Students'] / combined_df['NUMBER OF RETURNS']

# Funding Source Percentages
combined_df['Federal Funding Percentage'] = (combined_df['Expenditures per Equivalent Pupil - Federal Funds'] / combined_df['Expenditures per Equivalent Pupil']) * 100
combined_df['State and Local Funding Percentage'] = (combined_df['Expenditures per Equivalent Pupil - State and Local Funds'] / combined_df['Expenditures per Equivalent Pupil']) * 100

# Student Demographic Percentages
combined_df['Percent of Economically Disadvantaged Students'] = (combined_df['Economic Disadvantage'] / combined_df['Total Students']) * 100
combined_df['Percent of Students with Disabilities'] = (combined_df['Students with Disabilities'] / combined_df['Total Students']) * 100
combined_df['Percent of American Indian or Alaskan Native'] = (combined_df['American Indian or Alaskan Native'] / combined_df['Total Students']) * 100
combined_df['Percent of Asian or Pacific Islander'] = (combined_df['Asian or Pacific Islander'] / combined_df['Total Students']) * 100
combined_df['Percent of Black, Non-Hispanic'] = (combined_df['Black, Non-Hispanic'] / combined_df['Total Students']) * 100
combined_df['Percent of English Learner'] = (combined_df['English Learner'] / combined_df['Total Students']) * 100
combined_df['Percent of Hispanic'] = (combined_df['Hispanic'] / combined_df['Total Students']) * 100
combined_df['Percent of Migrant'] = (combined_df['Migrant'] / combined_df['Total Students']) * 100
combined_df['Percent of Multiracial'] = (combined_df['Multiracial'] / combined_df['Total Students']) * 100
combined_df['Percent of White, Non-Hispanic'] = (combined_df['White, Non-Hispanic'] / combined_df['Total Students']) * 100

In [136]:
columns_to_drop = [
    'Four Year Graduation Rate Numerator - Class of 2023',
    'Four Year Graduation Rate Denominator - Class of 2023',
    'Five Year Graduation Rate Numerator - Class of 2022',
    'Five Year Graduation Rate Denominator - Class of 2022',

    'Real Property Taxes Charged',
    'Public Utility Tangible Property Taxable Value',
    'Public Utility Tangible Property Taxes Levied',
    'Total Value',
    'Total Taxes',
    'Real Property',
    'Public Utility Tangible Property',
    'Amount',
    'TOTAL FEDERAL ADJUSTED GROSS INCOME',
    'TOTAL OHIO ADJUSTED GROSS INCOME',
    'TOTAL OHIO BUISNESS INCOME DEDUCTION',
    'TOTAL OHIO INCOME TAX BASE',
    'TOTAL OHIO INCOME TAX LIABILITY',

    'ADJUSTED GROSS INCOME RANK',
    'MEDIAN FEDERAL ADJUSTED GROSS INCOME RANK',
    'AVERAGE OHIO ADJUSTED GROSS INCOME RANK',
    'MEDIAN OHIO ADJUSTED GROSS INCOME RANK',

    'NUMBER OF PERSONAL EXEMPTIONS',
    'NUMBER OF SENIOR CITIZEN CREDITS'
]

combined_df.drop(columns=columns_to_drop,inplace=True)
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 606 entries, 442 to 139303
Data columns (total 62 columns):
 #   Column                                                         Non-Null Count  Dtype  
---  ------                                                         --------------  -----  
 0   District Name                                                  606 non-null    object 
 1   County                                                         606 non-null    object 
 2   Region                                                         606 non-null    object 
 3   Progress Component Star Rating                                 606 non-null    int64  
 4   Overall Composite                                              606 non-null    float64
 5   Overall Effect Size                                            606 non-null    float64
 6   Graduation Rate Component Rating                               606 non-null    int64  
 7   Graduation Rate Component Percent (Weighted Graduation Rate)  

In [137]:
# --- Create the Taxpayer Efficiency Score ---

# 1. Normalize the input variables to a 0-1 scale
# Normalize Median Income
min_income = combined_df['MEDIAN OHIO ADJUSTED GROSS INCOME'].min()
max_income = combined_df['MEDIAN OHIO ADJUSTED GROSS INCOME'].max()
combined_df['Normalized Median Income'] = (combined_df['MEDIAN OHIO ADJUSTED GROSS INCOME'] - min_income) / (max_income - min_income)

# Normalize Total Local Tax Burden
min_tax = combined_df['Total Local Tax Burden'].min()
max_tax = combined_df['Total Local Tax Burden'].max()
combined_df['Normalized Tax Burden'] = (combined_df['Total Local Tax Burden'] - min_tax) / (max_tax - min_tax)

# 2. Create the composite "Socio-Economic & Tax Burden Index"
# We give equal weight to both normalized components
combined_df['Socio-Economic & Tax Burden Index'] = (combined_df['Normalized Median Income'] + combined_df['Normalized Tax Burden']) / 2

# 3. Calculate the final Taxpayer Efficiency Score
combined_df['Taxpayer Efficiency Score'] = combined_df['Performance Index Score 2023-2024'] / (1 + combined_df['Socio-Economic & Tax Burden Index'])

# You can now view the most efficient districts
print("Top 15 Most Efficient School Districts:")
print(combined_df[['District Name', 'Taxpayer Efficiency Score', 'Performance Index Score 2023-2024', 'MEDIAN OHIO ADJUSTED GROSS INCOME', 'Total Local Tax Burden']].sort_values(by='Taxpayer Efficiency Score', ascending=False).head(15))

Top 15 Most Efficient School Districts:
                                            District Name  \
District IRN                                                
44826                                   Steubenville City   
45633                         Versailles Exempted Village   
49361                     Miller City-New Cleveland Local   
45955                                    New Bremen Local   
49783                                  Fort Loramie Local   
45963                                 New Knoxville Local   
48553                                        Marion Local   
48595                                 Fort Recovery Local   
45419         Hicksville Exempted Village School District   
46680                                   Tri-Village Local   
45948                                       Minster Local   
47225                                   West Geauga Local   
45328                         Columbiana Exempted Village   
47688                                   East 

In [138]:
# --- Create the Performance Opportunity Score (POS) ---

# 1. Normalize the core challenge indicators to a 0-1 scale
# Economic Disadvantage
min_ed = combined_df['Percent of Economically Disadvantaged Students'].min()
max_ed = combined_df['Percent of Economically Disadvantaged Students'].max()
combined_df['Normalized Econ Disadvantage'] = (combined_df['Percent of Economically Disadvantaged Students'] - min_ed) / (max_ed - min_ed)

# Median Income (already normalized in the previous step, but we'll ensure it's here)
min_income = combined_df['MEDIAN OHIO ADJUSTED GROSS INCOME'].min()
max_income = combined_df['MEDIAN OHIO ADJUSTED GROSS INCOME'].max()
combined_df['Normalized Median Income'] = (combined_df['MEDIAN OHIO ADJUSTED GROSS INCOME'] - min_income) / (max_income - min_income)

# 2. Create the weighted "Challenge Index"
# Giving 2/3 weight to economic disadvantage and 1/3 to the inverse of income capacity.
combined_df['Challenge Index'] = ((2 * combined_df['Normalized Econ Disadvantage']) + (1 - combined_df['Normalized Median Income'])) / 3

# 3. Calculate the "Expected Performance Score"
# This creates a sliding scale of expected performance based on the Challenge Index.
min_pi = combined_df['Performance Index Score 2023-2024'].min()
max_pi = combined_df['Performance Index Score 2023-2024'].max()
combined_df['Expected Performance'] = max_pi - (combined_df['Challenge Index'] * (max_pi - min_pi))

# 4. Calculate the final "Performance Opportunity Score"
combined_df['Performance Opportunity Score'] = combined_df['Performance Index Score 2023-2024'] - combined_df['Expected Performance']

# Display the top 15 districts that are over-performing the most
print("Top 15 Districts by Performance Opportunity Score (POS):")
print(combined_df[['District Name', 'Performance Opportunity Score', 'Performance Index Score 2023-2024', 'Expected Performance', 'Percent of Economically Disadvantaged Students']].sort_values(by='Performance Opportunity Score', ascending=False).head(15))

Top 15 Districts by Performance Opportunity Score (POS):
                              District Name  Performance Opportunity Score  \
District IRN                                                                 
44826                     Steubenville City                      54.037736   
50203                       Lordstown Local                      35.187085   
44248                   Logan-Hocking Local                      34.408908   
47928                   Dawson-Bryant Local                      32.399462   
47761                  Oak Hill Union Local                      31.492912   
44149          Ironton City School District                      30.321344   
442                        Manchester Local                      29.809194   
44461                      New Boston Local                      29.261364   
43828                        Coshocton City                      29.079755   
47969                   Symmes Valley Local                      28.774265   
50393  

In [139]:
import numpy as np

# --- Create the Next-Generation Value Score ---

# 1. Normalize all component metrics to a 0-1 scale

# Performance Metrics
min_pi = combined_df['Performance Index Score 2023-2024'].min()
max_pi = combined_df['Performance Index Score 2023-2024'].max()
combined_df['Normalized PI Score'] = (combined_df['Performance Index Score 2023-2024'] - min_pi) / (max_pi - min_pi)

min_att = combined_df['Total Attendance Percent'].min()
max_att = combined_df['Total Attendance Percent'].max()
combined_df['Normalized Attendance'] = (combined_df['Total Attendance Percent'] - min_att) / (max_att - min_att)

min_grad = combined_df['Four Year Graduation Rate - Class of 2023'].min()
max_grad = combined_df['Four Year Graduation Rate - Class of 2023'].max()
combined_df['Normalized Grad Rate'] = (combined_df['Four Year Graduation Rate - Class of 2023'] - min_grad) / (max_grad - min_grad)

# Cost of Living Metrics
min_inc = combined_df['MEDIAN OHIO ADJUSTED GROSS INCOME'].min()
max_inc = combined_df['MEDIAN OHIO ADJUSTED GROSS INCOME'].max()
combined_df['Normalized Median Income'] = (combined_df['MEDIAN OHIO ADJUSTED GROSS INCOME'] - min_inc) / (max_inc - min_inc)

min_tax = combined_df['Total Local Tax Burden'].min()
max_tax = combined_df['Total Local Tax Burden'].max()
combined_df['Normalized Tax Burden'] = (combined_df['Total Local Tax Burden'] - min_tax) / (max_tax - min_tax)

# 2. Create the Composite Scores

# Create the Holistic Performance Score with 3:2:1 weighting (PI:Grad:Att)
combined_df['Holistic Performance Score'] = ((3 * combined_df['Normalized PI Score']) + (2 * combined_df['Normalized Grad Rate']) + (1 * combined_df['Normalized Attendance'])) / 6

# Create the Cost of Living Index (equal weight)
combined_df['Cost of Living Index'] = (combined_df['Normalized Median Income'] + combined_df['Normalized Tax Burden']) / 2

# 3. Calculate the Final, Refined Value Score
# Using square root to dampen the effect of the denominator
combined_df['Final Value Score'] = (combined_df['Holistic Performance Score'] / (np.sqrt(combined_df['Cost of Living Index']) + 0.1)) * 100

# Display the top 15 districts with the new, more balanced score
print("Top 15 Districts by Final Value Score:")
print(combined_df[['District Name', 'Final Value Score', 'Performance Index Score 2023-2024', 'Four Year Graduation Rate - Class of 2023', 'MEDIAN OHIO ADJUSTED GROSS INCOME']].sort_values(by='Final Value Score', ascending=False).head(15))

Top 15 Districts by Final Value Score:
                                District Name  Final Value Score  \
District IRN                                                       
44826                       Steubenville City         195.242645   
50096                  Bloomfield-Mespo Local         176.830331   
45633             Versailles Exempted Village         168.350631   
49361         Miller City-New Cleveland Local         166.313073   
45963                     New Knoxville Local         163.979064   
48595                     Fort Recovery Local         163.385139   
46649                   Franklin Monroe Local         163.234772   
45955                        New Bremen Local         162.873745   
50542                            Dalton Local         161.972548   
49312                    Columbus Grove Local         161.877614   
49783                      Fort Loramie Local         161.871401   
46680                       Tri-Village Local         160.948221   
45328    

In [140]:
import numpy as np

# --- Create the Comprehensive Value Score (Adjusted for Size) ---

# 1. Normalize all component metrics to a 0-1 scale
# Performance Metrics (if not already done)
min_pi = combined_df['Performance Index Score 2023-2024'].min()
max_pi = combined_df['Performance Index Score 2023-2024'].max()
combined_df['Normalized PI Score'] = (combined_df['Performance Index Score 2023-2024'] - min_pi) / (max_pi - min_pi)

min_att = combined_df['Total Attendance Percent'].min()
max_att = combined_df['Total Attendance Percent'].max()
combined_df['Normalized Attendance'] = (combined_df['Total Attendance Percent'] - min_att) / (max_att - min_att)

min_grad = combined_df['Four Year Graduation Rate - Class of 2023'].min()
max_grad = combined_df['Four Year Graduation Rate - Class of 2023'].max()
combined_df['Normalized Grad Rate'] = (combined_df['Four Year Graduation Rate - Class of 2023'] - min_grad) / (max_grad - min_grad)

# Cost of Living & Complexity Metrics
min_inc = combined_df['MEDIAN OHIO ADJUSTED GROSS INCOME'].min()
max_inc = combined_df['MEDIAN OHIO ADJUSTED GROSS INCOME'].max()
combined_df['Normalized Median Income'] = (combined_df['MEDIAN OHIO ADJUSTED GROSS INCOME'] - min_inc) / (max_inc - min_inc)

min_tax = combined_df['Total Local Tax Burden'].min()
max_tax = combined_df['Total Local Tax Burden'].max()
combined_df['Normalized Tax Burden'] = (combined_df['Total Local Tax Burden'] - min_tax) / (max_tax - min_tax)

min_students = combined_df['Total Students'].min()
max_students = combined_df['Total Students'].max()
combined_df['Normalized Total Students'] = (combined_df['Total Students'] - min_students) / (max_students - min_students)


# 2. Create the Composite Scores
# Holistic Performance Score (50% PI, 33% Grad, 17% Att)
combined_df['Holistic Performance Score'] = ((3 * combined_df['Normalized PI Score']) + (2 * combined_df['Normalized Grad Rate']) + (1 * combined_df['Normalized Attendance'])) / 6

# Cost of Living Index
combined_df['Cost of Living Index'] = (combined_df['Normalized Median Income'] + combined_df['Normalized Tax Burden']) / 2

# Cost and Complexity Index (giving 2/3 weight to cost of living)
combined_df['Cost and Complexity Index'] = ((2 * combined_df['Cost of Living Index']) + (1 * combined_df['Normalized Total Students'])) / 3

# 3. Calculate the Final, Comprehensive Value Score
# Using square root to dampen the effect of the denominator
combined_df['Comprehensive Value Score'] = (combined_df['Holistic Performance Score'] / (np.sqrt(combined_df['Cost and Complexity Index']) + 0.1)) * 100

# Display the top 15 districts with the new, comprehensive score
print("Top 15 Districts by Comprehensive Value Score:")
print(combined_df[['District Name', 'Comprehensive Value Score', 'Holistic Performance Score', 'Cost and Complexity Index', 'Total Students']].sort_values(by='Comprehensive Value Score', ascending=False).head(100))
combined_df[['District Name', 'Comprehensive Value Score', 'Holistic Performance Score', 'Cost and Complexity Index', 'Total Students']].sort_values(by='Comprehensive Value Score', ascending=False).to_csv('top_schools.csv',index=False)

Top 15 Districts by Comprehensive Value Score:
                                District Name  Comprehensive Value Score  \
District IRN                                                               
44826                       Steubenville City                 209.712193   
50096                  Bloomfield-Mespo Local                 202.209391   
49361         Miller City-New Cleveland Local                 194.359193   
45633             Versailles Exempted Village                 194.071639   
45963                     New Knoxville Local                 191.613239   
...                                       ...                        ...   
46037           Eastern Local School District                 161.067335   
47514                         Riverdale Local                 160.746182   
49700                   Hopewell-Loudon Local                 160.557036   
48090                         Riverside Local                 160.489247   
44651                       Port Clinton 

In [141]:
# 1. Estimate the total market value of all real property
# In Ohio, taxable value is 35% of market value.
combined_df['Estimated Total Market Value'] = combined_df['Real Property Taxable Value (a)'] / 0.35

# 2. Calculate the estimated average home value using NUMBER OF RETURNS as a proxy for households
# We add 1 to the denominator to avoid any potential division by zero errors.
combined_df['Estimated Average Home Value'] = combined_df['Estimated Total Market Value'] / (combined_df['NUMBER OF RETURNS'] + 1)

# Now you can use 'Estimated Average Home Value' as the 'Median Home Value' in the final metric.
# Let's rename it for clarity and consistency with the previous metric's code.
combined_df.rename(columns={'Estimated Average Home Value': 'Median Home Value'}, inplace=True)


# Display the new column for a few districts
print(combined_df[['District Name', 'Median Home Value', 'Real Property Taxable Value (a)', 'NUMBER OF RETURNS']].head(30))

                                   District Name  Median Home Value  \
District IRN                                                          
442                             Manchester Local      131527.018080   
43489                                 Akron City      118915.487276   
43497                              Alliance City      124661.046427   
43505                               Ashland City      162188.956532   
43513                        Ashtabula Area City      128097.997622   
43521                Athens City School District      189145.929600   
43539                             Barberton City      125301.473807   
43547                           Bay Village City      326596.628635   
43554                             Beachwood City      465804.928289   
43562                               Bedford City      186559.922956   
43570                             Bellaire Local      165354.537333   
43588                         Bellefontaine City      138428.843325   
43596 

In [142]:
import numpy as np
# Assuming 'combined_df' is your fully merged and cleaned DataFrame
# from the previous steps.

# --- Step 1: Calculate Estimated Average Home Value ---
# This step is now included directly in the final calculation flow.
combined_df['Estimated Total Market Value'] = combined_df['Real Property Taxable Value (a)'] / 0.35
combined_df['Median Home Value'] = combined_df['Estimated Total Market Value'] / (combined_df['NUMBER OF RETURNS'] + 1)


# --- Step 2: Create the Affordability & Value Score ---

# A. Calculate the Housing Affordability Ratio
# This is the core of the new cost metric.
combined_df['Housing Affordability Ratio'] = combined_df['Median Home Value'] / (combined_df['MEDIAN OHIO ADJUSTED GROSS INCOME'] + 1)

# B. Normalize all component metrics to a 0-1 scale
# Performance Metrics
min_pi = combined_df['Performance Index Score 2023-2024'].min()
max_pi = combined_df['Performance Index Score 2023-2024'].max()
combined_df['Normalized PI Score'] = (combined_df['Performance Index Score 2023-2024'] - min_pi) / (max_pi - min_pi)

min_att = combined_df['Total Attendance Percent'].min()
max_att = combined_df['Total Attendance Percent'].max()
combined_df['Normalized Attendance'] = (combined_df['Total Attendance Percent'] - min_att) / (max_att - min_att)

min_grad = combined_df['Four Year Graduation Rate - Class of 2023'].min()
max_grad = combined_df['Four Year Graduation Rate - Class of 2023'].max()
combined_df['Normalized Grad Rate'] = (combined_df['Four Year Graduation Rate - Class of 2023'] - min_grad) / (max_grad - min_grad)

# Affordability & Complexity Metrics
min_afford = combined_df['Housing Affordability Ratio'].min()
max_afford = combined_df['Housing Affordability Ratio'].max()
combined_df['Normalized Affordability'] = (combined_df['Housing Affordability Ratio'] - min_afford) / (max_afford - min_afford)

min_tax = combined_df['Total Local Tax Burden'].min()
max_tax = combined_df['Total Local Tax Burden'].max()
combined_df['Normalized Tax Burden'] = (combined_df['Total Local Tax Burden'] - min_tax) / (max_tax - min_tax)

min_students = combined_df['Total Students'].min()
max_students = combined_df['Total Students'].max()
combined_df['Normalized Total Students'] = (combined_df['Total Students'] - min_students) / (max_students - min_students)

# C. Create the Composite Scores
# Holistic Performance Score (50% PI, 33% Grad, 17% Att)
combined_df['Holistic Performance Score'] = ((3 * combined_df['Normalized PI Score']) + (2 * combined_df['Normalized Grad Rate']) + (1 * combined_df['Normalized Attendance'])) / 6

# Affordability & Complexity Index (50% Affordability, 33% Tax, 17% Size)
combined_df['Affordability & Complexity Index'] = ((3 * combined_df['Normalized Affordability']) + (2 * combined_df['Normalized Tax Burden']) + (1 * combined_df['Normalized Total Students'])) / 6

# D. Calculate the Final Score
# Squaring the performance score gives it significantly more weight.
combined_df['Affordability & Value Score'] = (np.power(combined_df['Holistic Performance Score'], 2) / (np.sqrt(combined_df['Affordability & Complexity Index']) + 0.1)) * 100

# E. Display the final results
print("Top 15 Districts by Affordability & Value Score:")
print(combined_df[['District Name', 'Affordability & Value Score', 'Performance Index Score 2023-2024', 'Median Home Value', 'Total Students']].sort_values(by='Affordability & Value Score', ascending=False).head(15))

Top 15 Districts by Affordability & Value Score:
                                District Name  Affordability & Value Score  \
District IRN                                                                 
45955                        New Bremen Local                   240.112123   
49346                            Kalida Local                   232.162580   
49783                      Fort Loramie Local                   229.970510   
49361         Miller City-New Cleveland Local                   227.294309   
44826                       Steubenville City                   223.939103   
45633             Versailles Exempted Village                   221.265624   
49387                         Ottoville Local                   212.739386   
48637                            Newton Local                   210.557068   
49593                      Bloom-Vernon Local                   208.733166   
48553                            Marion Local                   207.573796   
45948          

In [147]:
import numpy as np

# Assuming 'combined_df' is your fully merged and cleaned DataFrame.

# --- Step 1: Create the Input Data ---

# Estimate the total market value of all real property (taxable value is 35% of market value in Ohio)
combined_df['Estimated Total Market Value'] = combined_df['Real Property Taxable Value (a)'] / 0.35

# Estimate the average home value using NUMBER OF RETURNS as a proxy for households
combined_df['Median Home Value'] = combined_df['Estimated Total Market Value'] / (combined_df['NUMBER OF RETURNS'] + 1)

# Calculate the Housing Affordability Ratio
combined_df['Housing Affordability Ratio'] = combined_df['Median Home Value'] / (combined_df['MEDIAN OHIO ADJUSTED GROSS INCOME'] + 1)


# --- Step 2: Normalize All Component Metrics (0-1 Scale) ---

# Performance Metrics
min_pi = combined_df['Performance Index Score 2023-2024'].min()
max_pi = combined_df['Performance Index Score 2023-2024'].max()
combined_df['Normalized PI Score'] = (combined_df['Performance Index Score 2023-2024'] - min_pi) / (max_pi - min_pi)

min_att = combined_df['Total Attendance Percent'].min()
max_att = combined_df['Total Attendance Percent'].max()
combined_df['Normalized Attendance'] = (combined_df['Total Attendance Percent'] - min_att) / (max_att - min_att)

min_grad = combined_df['Four Year Graduation Rate - Class of 2023'].min()
max_grad = combined_df['Four Year Graduation Rate - Class of 2023'].max()
combined_df['Normalized Grad Rate'] = (combined_df['Four Year Graduation Rate - Class of 2023'] - min_grad) / (max_grad - min_grad)

# Affordability & Complexity Metrics
min_afford = combined_df['Housing Affordability Ratio'].min()
max_afford = combined_df['Housing Affordability Ratio'].max()
combined_df['Normalized Affordability'] = (combined_df['Housing Affordability Ratio'] - min_afford) / (max_afford - min_afford)

min_tax = combined_df['Total Local Tax Burden'].min()
max_tax = combined_df['Total Local Tax Burden'].max()
combined_df['Normalized Tax Burden'] = (combined_df['Total Local Tax Burden'] - min_tax) / (max_tax - min_tax)

min_students = combined_df['Total Students'].min()
max_students = combined_df['Total Students'].max()
combined_df['Normalized Total Students'] = (combined_df['Total Students'] - min_students) / (max_students - min_students)


# --- Step 3: Create the Composite Scores ---

# Holistic Performance Score (50% PI, 33% Grad, 17% Att)
combined_df['Holistic Performance Score'] = ((3 * combined_df['Normalized PI Score']) + (2 * combined_df['Normalized Grad Rate']) + (1 * combined_df['Normalized Attendance'])) / 6

# Affordability & Complexity Index (50% Affordability, 33% Tax, 17% Size)
combined_df['Affordability & Complexity Index'] = ((3 * combined_df['Normalized Affordability']) + (2 * combined_df['Normalized Tax Burden']) + (1 * combined_df['Normalized Total Students'])) / 6


# --- Step 4: Calculate the Final Score ---

# Squaring the performance score gives it significantly more weight.
combined_df['Affordability & Value Score'] = (np.power(combined_df['Holistic Performance Score'], 2) / (np.sqrt(combined_df['Affordability & Complexity Index']) + 0.1)) * 100


# --- Step 5: Display the Final Results ---

print("Top 15 Districts by Affordability & Value Score:")
print(combined_df[['District Name', 'Affordability & Value Score', 'Performance Index Score 2023-2024', 'Median Home Value', 'Total Students']].sort_values(by='Affordability & Value Score', ascending=False).head(15))
combined_df[['District Name','County', 'Affordability & Value Score', 'Performance Index Score 2023-2024', 'Median Home Value', 'MEDIAN OHIO ADJUSTED GROSS INCOME','Total Local Tax Burden','Total Students']].sort_values(by='Affordability & Value Score', ascending=False).to_csv('top_schools.csv',index=False)

Top 15 Districts by Affordability & Value Score:
                                District Name  Affordability & Value Score  \
District IRN                                                                 
45955                        New Bremen Local                   240.112123   
49346                            Kalida Local                   232.162580   
49783                      Fort Loramie Local                   229.970510   
49361         Miller City-New Cleveland Local                   227.294309   
44826                       Steubenville City                   223.939103   
45633             Versailles Exempted Village                   221.265624   
49387                         Ottoville Local                   212.739386   
48637                            Newton Local                   210.557068   
49593                      Bloom-Vernon Local                   208.733166   
48553                            Marion Local                   207.573796   
45948          

In [148]:
# rank based on real estate prices and incomes

combined_df[['District Name','County', 'Affordability & Value Score', 'Performance Index Score 2023-2024', 'Median Home Value', 'MEDIAN OHIO ADJUSTED GROSS INCOME','Total Local Tax Burden','Total Students']].sort_values(by='Median Home Value', ascending=False).to_csv('top_real_estate.csv',index=False)
combined_df[['District Name','County', 'Affordability & Value Score', 'Performance Index Score 2023-2024', 'Median Home Value', 'MEDIAN OHIO ADJUSTED GROSS INCOME','Total Local Tax Burden','Total Students']].sort_values(by='MEDIAN OHIO ADJUSTED GROSS INCOME', ascending=False).to_csv('top_income.csv',index=False)